# Tagging multilingual texts
`flair` provides as multilingual POS model which includes the languages German and French. They report a high accuracy across languages. Therefore, it used here to tag the source languages which are not English or Arabic.

First, the necessary packages are installed and imported.


In [ ]:
%%bash
pip install flair
pip install nltk
pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for mpld3: filename=mpld3-0.3-py3-none-any.whl size=116702 sha256=dbe115c83ac0b92ecff6a94e2afff60b14a99c38d741624a895af3a698e3ab32
  Stored in directory: /root/.cache/pip/wheels/26/70/6a/1c79e59951a41b4045497da187b2724f5659ca64033cf4548e
  Created wheel for overrides: filename=overrides-3.1.0-py3-none-any.whl size=10187 sha256=211fdc10b1fff44422c352d4873f386ae9663d2e87ea73a65e7ed0644d5d6671
  Stored in directory: /root/.cache/pip/wheels/3a/0d/38/01a9bc6e20dcfaf0a6a7b552d03137558ba1c38aea47644682
  Created wheel for sqlitedict: filename=sqlitedict-2.0.0-py3-none-any.whl size=15736 sha256=34bb9857ef8210cef15dac95b94bac36fc1350c7988fe56f8dcca43aae2e573e
  Stored in directory: /root/.cache/pip/wheels/96/dd/2e/0ed4a25cb73fc30c7ea8d10b50acb7226175736067e40a7ea3
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=5022b6115d8feb5f5841e35c1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.4.1 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 3.10.1 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import os
from collections import defaultdict
import pickle


from flair.data import Sentence
from flair.models import SequenceTagger
from google.colab import drive
from tqdm import tqdm
drive.mount('/content/drive')

Mounted at /content/drive


Next, we define the necessary paths and constants.

In [ ]:
# Change the path to where the parallel data files are located on your device/drive
DATA_PATH = ["drive/MyDrive/Data/parallel/Tanzil-de-ha.txt",
             "drive/MyDrive/Data/parallel/Tanzil-fr-ha.txt"]

# These are the paths where the output should be stored.
OUT_PATH = ["/content/drive/MyDrive/Data/tagged/Tanzil-de-ha.tagged",
            "/content/drive/MyDrive/Data/tagged/Tanzil-fr-ha.tagged"]

# This the tag used for unknown words.
UNK = "<unk>"

Next, the tagger is initiated. We use the multilingual model which performs POS-Tagging for 12 languages.

In [ ]:
# Load tagger
tagger = SequenceTagger.load("flair/upos-multi")

/usr/local/lib/python3.7/dist-packages/huggingface_hub/file_download.py:563: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  FutureWarning,


Downloading:   0%|          | 0.00/314M [00:00<?, ?B/s]

2022-07-21 12:42:51,322 loading file /root/.flair/models/upos-multi/1a44f168663182024fd3ea6d7dcaeee47fe5bcb537cc737ad058b64ad4db9736.5f899f25846741510a6567b89027d988bd6f634b2776a7c3e834fea4629367cb
2022-07-21 12:42:51,734 SequenceTagger predicts: Dictionary with 21 tags: <unk>, O, PROPN, PUNCT, ADJ, NOUN, VERB, DET, ADP, AUX, PRON, PART, SCONJ, NUM, ADV, CCONJ, X, INTJ, SYM, <START>, <STOP>


Additionally, we define some function to load and save temporary results that are stored in the same directory as the output files.

In [ ]:
def load_pickle(file_name):
  with open(file_name, "rb") as file:
    return pickle.load(file)

def save_pickle(obj, file_name):
  with open(file_name, "wb") as file:
    pickle.dump(obj, file)

Now, Tagging is peformed. We iterate over all file in `DATA_PATH`, which contains lines in the following format: `source language ||| target language`. The left side is extracted and tagged with multingual POS flair model. 

In [ ]:
for file, out in zip(DATA_PATH, OUT_PATH):
  print(file)
  tmp_path = "{}.pickle".format(out.split(".")[0])
  print(tmp_path)
  with open(file, encoding="utf-8") as f:
    # Store sentences that have been tagged before.
    tag_dict = dict()
    start = 0 # Line to start at.
    try:
      line_dict = load_pickle(tmp_path)
      if line_dict.keys():
        start = max(line_dict.keys())
    except (FileNotFoundError, EOFError) as err:
      # Store results.
      print("No saved results found. Starting from scratch.")
      line_dict = defaultdict(list)
    for idx, line in tqdm(enumerate(f)):
      if idx < start:
        continue
      if idx%1000 == 0:
        save_pickle(line_dict, tmp_path)
      # Split line at delimiter
      line = line.split("|||")
      # Only look at line with the right format.
      if len(line) != 2:
        print("ERROR in line {}".format(idx))
        continue
      src, _ = line
      words = src.split()
      # Default is unknown tag for each word.
      tags = [(UNK, 0) for i in range(len(words))]
      sentence = Sentence(src, use_tokenizer=False)
      # If sentence has been tagged before, we don't need to tag it again.
      if src in tag_dict:
        tags = tag_dict[src]
      else:
        tagger.predict(sentence)
        # Store tag and tag confidence.
        tags = [(tags["value"], tags["confidence"]) for tags in sentence.to_dict()["all labels"]]
      line_dict[idx] = tags
      tag_dict[src] = tags
  print("Writing results to file...")
  # Write results to file.
  with open(out, "w", encoding="utf-8") as out_file:
    for i in range(len(line_dict)):
      tags = line_dict[i]
      tag_str = ["{}-{}".format(tag, score) for tag, score in tags]
      out_file.write("{}\n".format(" ".join(tag_str)))

drive/MyDrive/Data/parallel/Tanzil-de-ha.txt
/content/drive/MyDrive/Data/tagged/Tanzil-de-ha.pickle


46434it [20:24, 37.93it/s]


Writing results to file...
drive/MyDrive/Data/parallel/Tanzil-fr-ha.txt
/content/drive/MyDrive/Data/tagged/Tanzil-fr-ha.pickle


11762it [05:11, 37.80it/s]


Writing results to file...
